<a href="https://colab.research.google.com/github/Amikuto/DAaML/blob/master/06_CNN_embeddings_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [368]:
import pandas as pd
import numpy as np
import nltk
import torch
import torch.nn as nn

# 6. Классификация текстов при помощи сверточных сетей

__Автор__: Никита Владимирович Блохин (NVBlokhin@fa.ru)

Финансовый университет, 2020 г. 

## 1. Представление и предобработка текстовых данных в виде последовательностей

1.1 Представьте первое предложение из строки `text` как последовательность из индексов слов, входящих в это предложение

In [369]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [370]:
first_sen = nltk.sent_tokenize(text)[0].replace(".", "").lower()
first_sen

'select your preferences and run the install command'

In [371]:
words = nltk.word_tokenize(first_sen)
words

['select', 'your', 'preferences', 'and', 'run', 'the', 'install', 'command']

In [372]:
words_dict = {k: v for v, k in enumerate(words)}
words_dict

{'select': 0,
 'your': 1,
 'preferences': 2,
 'and': 3,
 'run': 4,
 'the': 5,
 'install': 6,
 'command': 7}

In [373]:
[words_dict[i] for i in words]

[0, 1, 2, 3, 4, 5, 6, 7]

1.2 Представьте первое предложение из строки `text` как последовательность векторов, соответствующих индексам слов. Для представления индекса в виде вектора используйте унитарное кодирование. В результате должен получиться двумерный тензор размера `количество слов в предложении` x `количество уникальных слов`

In [374]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [375]:
all_words = nltk.word_tokenize(text.lower().replace(".", ""))
all_words_dict = {k: v for v, k in enumerate(all_words)}
all_words_dict

{'select': 0,
 'your': 1,
 'preferences': 2,
 'and': 14,
 'run': 4,
 'the': 10,
 'install': 6,
 'command': 7,
 'stable': 8,
 'represents': 9,
 'most': 11,
 'currently': 12,
 'tested': 13,
 'supported': 15,
 'version': 16,
 'of': 17,
 'pytorch': 18,
 'note': 19,
 'that': 20,
 'libtorch': 21,
 'is': 22,
 'only': 23,
 'available': 24,
 'for': 25,
 'c++': 26}

In [376]:
first_sen = nltk.sent_tokenize(text)[0].replace(".", "").lower()
first_sen_words = nltk.word_tokenize(first_sen)
first_sen_words

['select', 'your', 'preferences', 'and', 'run', 'the', 'install', 'command']

In [377]:
tensor = torch.zeros(len(first_sen_words), len(all_words_dict))

In [378]:
for i, word in enumerate(first_sen_words):
  tensor[i][all_words_dict[word]] = 1
  print(all_words_dict[word])

0
1
2
14
4
10
6
7


In [379]:
tensor

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.]])

In [380]:
tensor.shape

torch.Size([8, 25])

1.3 Решите задачу 1.2, используя модуль `nn.Embedding`

In [381]:
from collections import Counter

vocab = Counter(all_words_dict)
vocab = sorted(vocab, key=vocab.get, reverse=True)
vocab_size = len(vocab)
vocab_size

25

In [382]:
word2idx = {word: ind for ind, word in enumerate(vocab)}
word2idx

{'c++': 0,
 'for': 1,
 'available': 2,
 'only': 3,
 'is': 4,
 'libtorch': 5,
 'that': 6,
 'note': 7,
 'pytorch': 8,
 'of': 9,
 'version': 10,
 'supported': 11,
 'and': 12,
 'tested': 13,
 'currently': 14,
 'most': 15,
 'the': 16,
 'represents': 17,
 'stable': 18,
 'command': 19,
 'install': 20,
 'run': 21,
 'preferences': 22,
 'your': 23,
 'select': 24}

In [383]:
encoded_sentences = [word2idx[word] for word in first_sen_words]
encoded_sentences

[24, 23, 22, 12, 21, 16, 20, 19]

In [384]:
emb_dim = 25
emb_layer = nn.Embedding(vocab_size, emb_dim)
word_vectors = emb_layer(torch.LongTensor(encoded_sentences))
word_vectors

tensor([[-9.9274e-03,  5.7113e-01,  8.8844e-02,  6.4107e-01, -6.9100e-01,
          5.0774e-02,  5.3088e-01, -1.0145e+00,  1.1685e+00,  1.1513e+00,
         -4.8536e-01,  1.0896e+00,  2.0024e-01,  6.1232e-01, -6.0658e-01,
         -1.3895e+00, -1.4070e+00, -3.2567e-01,  7.2541e-01, -4.7595e-01,
          7.7566e-02,  1.1994e+00,  5.2212e-01, -5.3060e-01, -4.8954e-01],
        [-3.1972e-01,  7.0856e-01, -5.9608e-02,  1.9033e-02, -5.7751e-01,
         -4.8397e-01, -2.1380e+00, -1.4264e-01, -1.9945e+00,  2.0475e+00,
         -3.1561e-01, -1.6744e+00,  1.5636e+00,  7.6315e-01, -1.0693e+00,
          1.6175e+00,  1.6690e+00, -1.4052e-02,  2.8299e-02,  1.1160e+00,
         -7.1009e-01,  4.8112e-01, -7.1232e-02,  1.1243e-01,  1.1346e-01],
        [-1.5913e+00, -8.7449e-01,  6.9500e-02,  2.2902e-01,  9.4500e-02,
         -1.3773e+00,  2.0051e-01,  7.4402e-02, -8.9629e-01, -7.6530e-01,
          3.1800e-01, -1.1697e+00, -2.2109e-01, -1.0762e-01,  8.1727e-02,
          4.0149e-01,  4.8609e-01,  

In [385]:
word_vectors.shape

torch.Size([8, 25])

## 2. Классификация фамилий по национальности (ConvNet)

Датасет: https://disk.yandex.ru/d/owHew8hzPc7X9Q?w=1

2.1 Считать файл `surnames/surnames.csv`. 

2.2 Закодировать национальности числами, начиная с 0.

2.3 Разбить датасет на обучающую и тестовую выборку

2.4 Реализовать класс `Vocab` (токен = __символ__)
  * добавьте в словарь специальный токен `<PAD>` с индексом 0
  * при создании словаря сохраните длину самой длинной последовательности из набора данных в виде атрибута `max_seq_len`

2.5 Реализовать класс `SurnamesDataset`
  * метод `__getitem__` возвращает пару: <последовательность индексов токенов (см. 1.1 ), номер класса> 
  * длина каждой такой последовательности должна быть одинаковой и равной `vocab.max_seq_len`. Чтобы добиться этого, дополните последовательность справа индексом токена `<PAD>` до нужной длины

2.6. Обучить классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`. Рассмотрите два варианта: 
    - когда токен представляется в виде унитарного вектора и модуль `nn.Embedding` не обучается
    - когда токен представляется в виде вектора небольшой размерности (меньше, чем размер словаря) и модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`

2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: прогнать несколько фамилий студентов группы через модели и проверить результат. Для каждой фамилии выводить 3 наиболее вероятных предсказания.

In [386]:
surname_dataset = pd.read_csv("./surnames/surnames.csv")
surname_dataset.head()

,surname,nationality
0,Woodford,English
1,Coté,French
2,Kore,English
3,Koury,Arabic
4,Lebzak,Russian


In [387]:
surname_dict = pd.Series(surname_dataset.nationality.unique()).to_dict()
surname_dict = dict(map(reversed, surname_dict.items()))
surname_dict_reverse = {v: k for k, v in surname_dict.items()}
surname_dict

{'English': 0,
 'French': 1,
 'Arabic': 2,
 'Russian': 3,
 'Japanese': 4,
 'Chinese': 5,
 'Italian': 6,
 'Czech': 7,
 'Irish': 8,
 'German': 9,
 'Greek': 10,
 'Spanish': 11,
 'Polish': 12,
 'Dutch': 13,
 'Vietnamese': 14,
 'Korean': 15,
 'Portuguese': 16,
 'Scottish': 17}

In [388]:
dataset_nation_as_index = surname_dataset.copy()
dataset_nation_as_index.nationality = surname_dataset.nationality.map(lambda x: surname_dict[x])
dataset_nation_as_index

,surname,nationality
0,Woodford,0
1,Coté,1
2,Kore,0
3,Koury,2
4,Lebzak,3
...,...,...
10975,Quraishi,2
10976,Innalls,0
10977,Król,12
10978,Purvis,0


In [389]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset_nation_as_index["surname"], dataset_nation_as_index["nationality"], test_size=0.2)

In [390]:
X_train

404      Yakunichev
2146      Ricchetti
6500        Taverna
10197    Karahalios
9552         Senior
            ...    
7937          Anton
9975            Lai
4349         Mohren
559         Brannon
555        Darchiev
Name: surname, Length: 8784, dtype: object

In [391]:
max(X_train.map(len))

17

In [392]:
y_train

404       3
2146      6
6500      6
10197    10
9552      0
         ..
7937      0
9975      5
4349     13
559       8
555       3
Name: nationality, Length: 8784, dtype: int64

In [393]:
class Vocab:
  def __init__(self, column: pd.DataFrame | pd.Series):
    all_chars = pd.Series(column.values).map(lambda x: list(x.lower())).explode().unique()
    all_chars = np.insert(all_chars, 0, "<PAD>")
    self.idx_to_token = {index: token for index, token in enumerate(all_chars)}
    self.token_to_idx = {token: index for index, token in enumerate(all_chars)}
    self.max_seq_len = max(column.map(len))

In [394]:
vocab = Vocab(dataset_nation_as_index["surname"])
vocab.max_seq_len

17

2.5 Реализовать класс `SurnamesDataset`
  * метод `__getitem__` возвращает пару: <последовательность индексов токенов (см. 1.1 ), номер класса>
  * длина каждой такой последовательности должна быть одинаковой и равной `vocab.max_seq_len`. Чтобы добиться этого, дополните последовательность справа индексом токена `<PAD>` до нужной длины

In [520]:
from torch.utils.data import Dataset


class SurnamesDataset(Dataset):
  def __init__(self, X, y, vocab: Vocab):
    self.X = X
    self.y = y
    self.vocab = vocab
    self.max_X = 17
    self.max_y = 10

  def vectorize(self, surename):
    tensor = torch.zeros(self.vocab.max_seq_len, dtype=torch.long)
    for i, val in enumerate(surename):
      tensor[i] = self.vocab.token_to_idx[val.lower()]
    tensor[tensor==0] = self.vocab.token_to_idx["<PAD>"]
    return tensor

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    surname = self.X.iloc[idx]

    nation_tensor = torch.zeros(len(surname_dict))
    nation = self.y.iloc[idx]
    nation_tensor[nation] = 1

    # return emb(torch.LongTensor(self.vectorize(surname))), nation
    return self.vectorize(surname), nation_tensor

In [521]:
dataset_train = SurnamesDataset(X=X_train, y=y_train, vocab=vocab)

In [522]:
X_train.iloc[0], y_train.iloc[0]

('Yakunichev', 3)

In [523]:
dataset_train[0]

(tensor([12, 16,  9, 11, 18, 17,  6, 19, 10, 22,  0,  0,  0,  0,  0,  0,  0]),
 tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))



2.6. Обучить классификатор.

  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`. Рассмотрите два варианта:
    - когда токен представляется в виде унитарного вектора и модуль `nn.Embedding` не обучается
    - когда токен представляется в виде вектора небольшой размерности (меньше, чем размер словаря) и модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`

In [527]:
# from argparse import Namespace
#
# args = Namespace(
#   # Data and path information
#   surname_csv="data/surnames/surnames_with_splits.csv",
#   vectorizer_file="vectorizer.json",
#   model_state_file="model.pth",
#   save_dir="model_storage/ch4/surname_mlp",
#   # Model hyper parameters
#   hidden_dim=300
# # Training  hyper parameters
# seed=1337,
# num_epochs=100,
# early_stopping_criteria=5,
# learning_rate=0.001,
# batch_size=64,
# # Runtime options omitted for space
# )

In [526]:
import torch.nn as nn
import torch.nn.functional as F

class SurnameClassifier(nn.Module):
  """ A 2-layer multilayer perceptron for classifying surnames """
  def __init__(self, input_dim, hidden_dim, output_dim):
    """
    Args:
        input_dim (int): the size of the input vectors
        hidden_dim (int): the output size of the first Linear layer
        output_dim (int): the output size of the second Linear layer
    """
    super(SurnameClassifier, self).__init__()
    self.fc1 = nn.Linear(input_dim, hidden_dim)
    self.fc2 = nn.Linear(hidden_dim, output_dim)

  def forward(self, x_in, apply_softmax=False):
    """The forward pass of the classifier

    Args:
        x_in (torch.Tensor): an input data tensor
            x_in.shape should be (batch, input_dim)
        apply_softmax (bool): a flag for the softmax activation
            should be false if used with the cross-entropy losses
    Returns:
        the resulting tensor. tensor.shape should be (batch, output_dim).
    """
    intermediate_vector = F.relu(self.fc1(x_in))
    prediction_vector = self.fc2(intermediate_vector)

    if apply_softmax:
      prediction_vector = F.softmax(prediction_vector, dim=1)

    return prediction_vector

In [528]:
from torch import optim

classifier = SurnameClassifier(input_dim=17,
                               hidden_dim=300,
                               output_dim=len(surname_dict))

loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)

In [531]:
from torch.utils.data import DataLoader

batch_size = 1
trainloader = DataLoader(
  dataset=dataset_train,
  batch_size=batch_size,
  # collate_fn=my_collate,
  # shuffle=True,
  num_workers=0
)

In [560]:
emb = nn.Embedding(len(vocab.token_to_idx) + 1, 18)
print(emb.weight.shape)
def get_embedding_index(x):
  results = torch.where(torch.sum((emb.weight==x), axis=1))
  if len(results[0])==len(x):
    return None
  else:
    return results[0][0]

torch.Size([57, 18])


In [580]:
dataset_train[2][0]

tensor([ 7, 16, 22, 10,  5, 18, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [582]:
emb(torch.LongTensor(dataset_train[2][0]))

tensor([[ 1.2176, -0.3957, -1.2966, -1.2530,  0.9588, -0.9008,  0.4871, -1.4253,
          1.1304,  0.0426,  0.9265, -0.6168, -0.4674,  0.2967, -0.7047, -1.4208,
          0.2573,  1.1751],
        [-2.1496,  0.1898, -1.3134,  0.8384,  0.5868,  0.2530, -0.5224,  2.8181,
         -0.4949,  0.1614, -0.3951, -0.1711,  1.3361, -0.6808, -0.4731,  0.5342,
         -0.5457,  0.1877],
        [-1.0207,  0.4781, -0.3857, -1.6426, -1.3560,  0.5167,  0.5013,  1.3497,
         -1.3976, -0.6116, -1.4255, -1.3122,  0.4944, -1.0234, -1.4764, -0.6822,
         -1.8695, -1.0606],
        [-0.6642,  1.4018, -0.2542, -1.5203,  0.7107, -1.2889,  0.9431,  1.2131,
          1.1953, -0.2241,  1.9249,  1.2686,  0.4294,  1.2645,  0.1171,  0.8292,
          0.7080,  1.1173],
        [-0.2245,  0.8528, -1.8789, -0.0196, -0.8057,  1.0572,  1.8532,  1.0886,
         -0.7508, -0.3876,  0.2266,  1.9919,  0.3684,  0.2896, -0.3405,  0.2992,
          1.3421, -1.6408],
        [-1.1875, -0.6919,  0.6618,  0.5580, -0.70

In [558]:
all_losses = []
for x, y in trainloader:
  y_pred = classifier.forward(x.to(torch.float32), False)
  y_item = y_pred.topk(1)[1].item()
  print(f"Predicted: {surname_dict_reverse[y_item]} Real: {surname_dict_reverse[torch.where(y)[1].item()]}")
  loss = loss_func(y_pred, y)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  all_losses.append(loss.item())
  # break

Predicted: Russian real: Russian
Predicted: Russian real: Italian
Predicted: English real: Italian
Predicted: Russian real: Greek
Predicted: English real: English
Predicted: Russian real: Japanese
Predicted: Chinese real: Chinese
Predicted: English real: English
Predicted: English real: English
Predicted: English real: English
Predicted: Russian real: English
Predicted: Russian real: Russian
Predicted: English real: English
Predicted: Arabic real: Arabic
Predicted: English real: Arabic
Predicted: Russian real: Russian
Predicted: English real: Japanese
Predicted: English real: Italian
Predicted: English real: English
Predicted: Russian real: Italian
Predicted: English real: French
Predicted: English real: Japanese
Predicted: English real: Italian
Predicted: Russian real: Japanese
Predicted: Chinese real: Chinese
Predicted: English real: Arabic
Predicted: English real: Italian
Predicted: English real: Russian
Predicted: English real: French
Predicted: English real: Irish
Predicted: Engli

In [589]:
all_losses = []
for x, y in trainloader:
  e = emb(torch.LongTensor(x))[0]
  print(e)
  y_pred = classifier.forward(e, False)
  # y_item = y_pred.topk(1)[1].item()
  # print(f"Predicted: {surname_dict_reverse[y_item]} Real: {surname_dict_reverse[torch.where(y)[1].item()]}")
  # loss = loss_func(y_pred, y)
  # optimizer.zero_grad()
  # loss.backward()
  # optimizer.step()
  # all_losses.append(loss.item())
  break

tensor([[ 0.4687, -0.2726,  0.4948,  0.7519, -0.9422, -0.2491,  0.2583,  0.1550,
          0.1373,  0.8164, -0.7138, -0.4107, -1.7546,  0.2560,  1.3780, -1.6772,
          1.2336,  0.4606],
        [-2.1496,  0.1898, -1.3134,  0.8384,  0.5868,  0.2530, -0.5224,  2.8181,
         -0.4949,  0.1614, -0.3951, -0.1711,  1.3361, -0.6808, -0.4731,  0.5342,
         -0.5457,  0.1877],
        [ 0.1176, -0.3931, -0.9586,  0.6160,  0.3768,  1.0142, -1.4214,  0.0523,
          1.5361, -0.0277,  1.4165,  0.0559, -1.3251,  0.8322, -0.9875,  0.5960,
          0.0479, -0.5808],
        [-1.2180,  1.3440, -0.2093,  0.7290,  1.0761, -0.9437,  0.4679,  0.0815,
          1.8332, -0.5795,  0.3734,  1.4527,  1.9830,  0.5449, -1.0593,  0.2126,
          1.4386, -0.8699],
        [-1.1875, -0.6919,  0.6618,  0.5580, -0.7013, -0.4408, -2.0232,  0.5878,
          0.9268,  0.6745, -0.0256,  0.1169,  0.9231,  1.2201,  0.0902, -0.2580,
         -0.7337,  0.4261],
        [ 0.6357,  0.6207,  0.1148, -1.5930,  0.32

In [ ]:
for epoch in range(20):
  for x, y in trainloader:
    output, loss = train(x[0], y)
    current_loss += loss
    guess, guess_i = categoryFromOutput(output)
    correct = '✓' if guess == y else f"✗ {categoryFromOutput(output[0])[0]}"
    all_losses.append(current_loss)
    current_loss = 0
  print(f"Epoch {epoch}: {np.array(all_losses).mean()}")

In [318]:
a = torch.nn.Embedding(10, 50)
b = torch.LongTensor([2,8])
results = a(b)



indices = torch.Tensor(list(map(get_embedding_index, results)))
indices

tensor([2., 8.])

In [319]:
results

tensor([[-0.4098,  0.2533,  1.2605,  0.7038,  0.2500,  0.9289, -0.2418,  0.5768,
          0.4933, -0.2881, -0.0395, -0.2379, -0.6801,  0.5488,  1.8835,  1.4410,
          1.0493, -0.1259, -0.1851, -1.4215,  0.8290, -0.2408,  0.3402, -1.4582,
          2.0032,  0.4850, -1.2842,  0.1348,  0.8905, -0.9637,  0.7791, -1.8000,
          0.6366,  0.2020,  1.3995, -0.6345, -0.4659, -1.3530,  0.7784,  0.5687,
          0.2248, -1.1422,  0.2366, -0.8745,  1.5936, -2.4422,  0.9036, -0.2850,
         -0.9802,  0.4497],
        [ 0.3270, -1.1546, -1.2967, -0.5088, -1.7634,  0.0211,  0.9224,  0.2901,
          0.4979,  1.0452,  2.3358, -0.8311,  0.7015,  0.2112,  0.7016, -0.0864,
          0.6148, -1.3134,  0.2297,  1.2282, -1.1078, -1.7340,  0.4388,  0.1766,
          0.0893,  2.1184,  0.7886,  0.2694, -0.0800,  0.5345,  0.9193,  1.6917,
         -0.5858,  1.1435,  0.3770, -0.0127,  0.5832,  1.5580,  1.7289,  1.7571,
         -0.0380, -0.7875, -1.8100, -1.1700,  0.6741,  1.6696,  0.3956, -1.6179,


In [273]:
import torch.nn as nn

class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(RNN, self).__init__()

    self.hidden_size = hidden_size

    self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
    self.i2o = nn.Linear(input_size + hidden_size, output_size)
    self.softmax = nn.LogSoftmax(dim=1)

  def forward(self, input, hidden):
    combined = torch.cat((input, hidden), 1)
    hidden = self.i2h(combined)
    output = self.i2o(combined)
    output = self.softmax(output)
    return output, hidden

  def initHidden(self):
    return torch.zeros(1, self.hidden_size)

In [274]:
n_hidden = 128
n_categories = len(surname_dict)
rnn = RNN(dataset_train.vocab.max_seq_len, n_hidden, n_categories)

In [275]:
criterion = nn.NLLLoss()

In [276]:
learning_rate = 0.005

optimizer = torch.optim.SGD(rnn.parameters(), lr=learning_rate, momentum=0.4)

def train(line_tensor, category_tensor):
  hidden = rnn.initHidden()
  rnn.zero_grad()

  # torch.tensor(line_tensor, dtype=torch.long)
  for i in range(torch.tensor(line_tensor, dtype=torch.long).size()[0]):
    output, hidden = rnn(line_tensor[i], hidden)

  loss = criterion(output, category_tensor)
  # loss.backward()
  # for p in rnn.parameters():
  #     p.data.add_(p.grad.data, alpha=-learning_rate)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  return output, loss.item()

In [277]:
from torch.utils.data import DataLoader

batch_size = 1
trainloader = DataLoader(
  dataset=dataset_train,
  # batch_size=batch_size,
  # collate_fn=my_collate,
  shuffle=True,
  num_workers=0
)
# testloader = DataLoader(
#   dataset=snds_test,
#   # batch_size=batch_size,
#   # collate_fn=my_collate,
#   shuffle=False,
#   num_workers=0
# )

In [279]:
def categoryFromOutput(output):
  top_n, top_i = output.topk(1)
  category_i = top_i[0].item()
  return surname_dict[category_i], category_i

In [282]:
current_loss = 0
all_losses = []

n_iters = 1000
print_every = 100
plot_every = 100

for epoch in range(20):
  for x, y in trainloader:
    print(x)
    break
  break
  #   output, loss = train(x[0], y)
  #   current_loss += loss
  #   guess, guess_i = categoryFromOutput(output)
  #   correct = '✓' if guess == y else f"✗ {categoryFromOutput(output[0])[0]}"
  #   all_losses.append(current_loss)
  #   current_loss = 0
  # print(f"Epoch {epoch}: {np.array(all_losses).mean()}")

[tensor([5]), tensor([17]), tensor([24]), tensor([14]), tensor([12]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0])]


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class MultilayerPerceptron(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    """
    Args:
        input_dim (int): the size of the input vectors
        hidden_dim (int): the output size of the first Linear layer
        output_dim (int): the output size of the second Linear layer
    """
    super(MultilayerPerceptron, self).__init__()
    self.fc1 = nn.Linear(input_dim, hidden_dim)
    self.fc2 = nn.Linear(hidden_dim, output_dim)

  def forward(self, x_in, apply_softmax=False):
    """The forward pass of the MLP

    Args:
        x_in (torch.Tensor): an input data tensor
            x_in.shape should be (batch, input_dim)
        apply_softmax (bool): a flag for the softmax activation
            should be false if used with the cross-entropy losses
    Returns:
        the resulting tensor. tensor.shape should be (batch, output_dim)
    """
    intermediate = F.relu(self.fc1(x_in))
    output = self.fc2(intermediate)

    if apply_softmax:
      output = F.softmax(output, dim=1)
    return output

In [ ]:
batch_size = 2 # number of samples input at once
input_dim = 17
hidden_dim = 50
output_dim = len(surname_dict)

# Initialize model
mlp = MultilayerPerceptron(input_dim, hidden_dim, output_dim)
print(mlp)

In [ ]:
y_output = mlp(x_input, apply_softmax=False)
describe(y_output)

## 3. Классификация обзоров на фильмы (ConvNet)

Датасет: https://disk.yandex.ru/d/tdinpb0nN_Dsrg

2.1 Создайте набор данных на основе файлов polarity/positive_reviews.csv (положительные отзывы) и polarity/negative_reviews.csv (отрицательные отзывы). Разбейте на обучающую и тестовую выборку.
  * токен = __слово__
  * данные для обучения в датасете представляются в виде последовательности индексов токенов
  * словарь создается на основе _только_ обучающей выборки. Для корректной обработки ситуаций, когда в тестовой выборке встретится токен, который не хранится в словаре, добавьте в словарь специальный токен `<UNK>`
  * добавьте предобработку текста

2.2. Обучите классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding` 
    - подберите адекватную размерность вектора эмбеддинга: 
    - модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`


2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: придумать небольшой отзыв, прогнать его через модель и вывести номер предсказанного класса (сделать это для явно позитивного и явно негативного отзыва)
* Целевое значение accuracy на валидации - 70+%

In [ ]:
positive_raw = pd.read_csv("./polarity/positive_reviews.csv", header=None)
negative_raw = pd.read_csv("./polarity/negative_reviews.csv", header=None)

In [ ]:
positive_raw.head()

In [ ]:
negative_raw.head()

In [ ]:
positive_raw["state"] = 1
negative_raw["state"] = 0

In [ ]:
negative_raw

In [ ]:
positive_raw

In [ ]:
all_reviews = pd.concat([negative_raw, positive_raw], axis=0)
all_reviews.columns = ["review", "rating"]
all_reviews

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(all_reviews["review"], all_reviews["rating"], test_size=0.2)

In [ ]:
nltk.word_tokenize(all_reviews.review)

In [ ]:
all_reviews["review"].flatten()